In [2]:
# save as: fetch_tmdb_data.py
import requests
import pandas as pd
import time

# =============================================================================
# YOUR API KEY HERE
# =============================================================================
API_KEY = "7807b8f8e5ea17c89e662c919e28d7bc"  # ← Paste your TMDB API key

# =============================================================================
# FUNCTION: Get movies from TMDB
# =============================================================================
def get_movies(page):
    """Fetch one page of movies from TMDB"""
    url = "https://api.themoviedb.org/3/discover/movie"
    params = {
        'api_key': API_KEY,
        'sort_by': 'popularity.desc',
        'vote_count.gte': 50,
        'primary_release_date.gte': '2020-01-01',
        'page': page
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"\nError fetching page {page}: {e}")
        return None

# =============================================================================
# FUNCTION: Get movie details with retry logic
# =============================================================================
def get_movie_details(movie_id, retries=3):
    """Get movie details with retry logic for connection errors"""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}"
    params = {
        'api_key': API_KEY,
        'append_to_response': 'credits'
    }
    
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            if attempt < retries - 1:
                print(f"\nRetrying movie {movie_id} (attempt {attempt + 2}/{retries})...")
                time.sleep(2)  # Wait 2 seconds before retry
            else:
                print(f"\nFailed to fetch movie {movie_id} after {retries} attempts")
                return None
    
    return None

# =============================================================================
# MAIN EXTRACTION
# =============================================================================
movies = []

print("Starting extraction... This will take about 5-7 minutes.")
print("(Slower requests to avoid connection errors)")
print("=" * 60)

for page in range(1, 26):  # 25 pages = ~500 movies
    print(f"Fetching page {page}/25...", end=' ')
    
    # Get movies from this page
    data = get_movies(page)
    
    if not data or 'results' not in data:
        print("Skipped (error)")
        continue
    
    print(f"({len(data['results'])} movies)")
    
    # For each movie, get full details
    for i, m in enumerate(data['results'], 1):
        print(f"  Movie {i}/{len(data['results'])}: {m['title'][:30]}...", end='\r')
        
        # Get detailed info including cast and crew
        details = get_movie_details(m['id'])
        
        if not details:
            continue
        
        # Extract genres (all of them)
        genres = [g['name'] for g in details.get('genres', [])]
        
        # Extract director
        director = None
        if 'credits' in details and 'crew' in details['credits']:
            director = next(
                (c['name'] for c in details['credits']['crew'] if c['job'] == 'Director'), 
                None
            )
        
        # Extract ALL cast members
        cast = []
        if 'credits' in details and 'cast' in details['credits']:
            cast = [c['name'] for c in details['credits']['cast']]
        
        # Build movie record
        movies.append({
            'tmdb_id': m['id'],
            'title': m['title'],
            'release_year': m['release_date'][:4] if m.get('release_date') else None,
            'release_date': m.get('release_date'),
            'runtime': details.get('runtime'),
            'budget': details.get('budget', 0),
            'revenue': details.get('revenue', 0),
            'vote_average': m.get('vote_average'),
            'vote_count': m.get('vote_count'),
            'popularity': m.get('popularity'),
            'genres': genres,
            'director': director,
            'cast': cast,
            'imdb_id': details.get('imdb_id'),
            'original_language': m.get('original_language'),
            'status': details.get('status')
        })
        
        # Longer delay to avoid connection reset (1 second per movie)
        time.sleep(1)
    
    print(f"  Page {page} complete! Total movies collected: {len(movies)}")
    
    # Save progress after each page (in case of errors)
    if len(movies) > 0:
        df_temp = pd.DataFrame(movies)
        df_temp.to_csv('tmdb_movies_progress.csv', index=False)

print("\n" + "=" * 60)

# =============================================================================
# SAVE FINAL CSV
# =============================================================================
if len(movies) > 0:
    df = pd.DataFrame(movies)
    df.to_csv('tmdb_movies.csv', index=False)
    
    print(f"✓ Done! {len(df)} movies saved to 'tmdb_movies.csv'")
    print("=" * 60)
    
    # Show preview
    print("\nData Preview:")
    print(df[['title', 'release_year', 'director', 'budget', 'revenue']].head())
    
    print("\nColumn Names:")
    print(df.columns.tolist())
    
    print("\nSummary:")
    print(f"  - Total Movies: {len(df)}")
    print(f"  - Date Range: {df['release_year'].min()} to {df['release_year'].max()}")
    print(f"  - Unique Directors: {df['director'].nunique()}")
    print(f"  - Movies with Budget Data: {df['budget'].gt(0).sum()}")
    print(f"  - Movies with Revenue Data: {df['revenue'].gt(0).sum()}")
    
    # Count total unique actors
    all_actors = []
    for cast_list in df['cast']:
        all_actors.extend(cast_list)
    print(f"  - Total Unique Actors: {len(set(all_actors))}")
else:
    print("No movies were collected. Please check your API key and internet connection.")

Starting extraction... This will take about 5-7 minutes.
(Slower requests to avoid connection errors)
Fetching page 1/25... (20 movies)
  Page 1 complete! Total movies collected: 20...
Fetching page 2/25... (20 movies)
  Page 2 complete! Total movies collected: 40..
Fetching page 3/25... (20 movies)
  Page 3 complete! Total movies collected: 60
Fetching page 4/25... (20 movies)
  Page 4 complete! Total movies collected: 80
Fetching page 5/25... (20 movies)
  Page 5 complete! Total movies collected: 100.
Fetching page 6/25... (20 movies)
  Movie 2/20: Flow...stance...
Retrying movie 823219 (attempt 2/3)...
  Page 6 complete! Total movies collected: 120..
Fetching page 7/25... (20 movies)
  Page 7 complete! Total movies collected: 140..
Fetching page 8/25... (20 movies)
  Page 8 complete! Total movies collected: 160..
Fetching page 9/25... (20 movies)
  Page 9 complete! Total movies collected: 180..
Fetching page 10/25... (20 movies)
  Page 10 complete! Total movies collected: 200.
Fetch

In [3]:
df = pd.read_csv('tmdb_movies_progress.csv')
print(df.head())

   tmdb_id                     title  release_year release_date  runtime  \
0  1062722              Frankenstein          2025   2025-10-17      150   
1  1054867  One Battle After Another          2025   2025-09-23      162   
2  1248226                  Playdate          2025   2025-11-05       95   
3  1116465                  A Legend          2024   2024-07-05      129   
4  1242898        Predator: Badlands          2025   2025-11-05      107   

      budget    revenue  vote_average  vote_count  popularity  \
0  120000000     144496         7.875        1453    582.9067   
1  130000000  200300000         7.606        1420    414.7432   
2          0          0         6.500         177    396.6845   
3          0   11242056         6.325          63    322.4436   
4  105000000  136304860         7.400         342    263.6485   

                                       genres               director  \
0              ['Drama', 'Horror', 'Fantasy']     Guillermo del Toro   
1       

In [4]:
import pandas as pd
import ast

# Load original
df = pd.read_csv('tmdb_movies_progress.csv')

# Fix lists
df['cast'] = df['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])
df['genres'] = df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [])

# Calculate ROI & Profit
df['ROI'] = (df['revenue'] - df['budget']) / df['budget'].replace(0, pd.NA)
df['profit'] = df['revenue'] - df['budget']
df['ROI'] = pd.to_numeric(df['ROI'], errors='coerce')
df['profit'] = pd.to_numeric(df['profit'], errors='coerce')
df['ROI'] = df['ROI'].fillna(0)

# Explode both cast and genres at the same time → one perfect row per actor + genre combo
df_exploded = df.explode('cast').explode('genres')

# Rename
df_exploded.rename(columns={'cast': 'actor', 'genres': 'genre'}, inplace=True)

# Reorder & select only useful columns
final_df = df_exploded[[
    'tmdb_id', 'title', 'release_year', 'release_date',
    'runtime', 'budget', 'revenue', 'profit', 'ROI',
    'vote_average', 'vote_count', 'popularity',
    'genre', 'director', 'actor',
    'original_language', 'status'
]]

# Save the ONE file you will use in Tableau
final_df.to_csv('TMDB_FINAL_ONE_FILE_FOR_TABLEAU.csv', index=False)
print("Done! File saved → TMDB_FINAL_ONE_FILE_FOR_TABLEAU.csv")
print(f"Total rows: {len(final_df):,}")

Done! File saved → TMDB_FINAL_ONE_FILE_FOR_TABLEAU.csv
Total rows: 50,729
